# Dark Sky API (Weather)

Answer the following questions using the [Dark Sky API](https://darksky.net/dev/). I've added three cells for each question but you're free to use more or less! Hold `Shift` and hit `Enter` to run a cell, and use the `+` on the top left to add a new cell to a notebook.

If you spend some time reading the documentation your life will probably be a little bit easier!

## 0) Import any libraries you might need

- *Tip: We're going to be downloading things from the internet, so we probably need `requests`.*

In [ ]:
import requests

In [ ]:
import statistics

In [ ]:
import datetime

## 1) Make a request from the Dark Sky API for where you were born (or lived, or want to visit!).

- *Tip: The URL we used in class was for a place near San Francisco. What was the format of the endpoint that made this happen?*
- *Tip: Save the URL as a separate variable, and be sure to not have `[` and `]` inside.*
- *Tip: How is north vs. south and east vs. west latitude/longitude represented? Is it the normal North/South/East/West?*
- *Tip: You know it's JSON, but Python doesn't! Make sure you aren't trying to deal with plain text.* 
- *Tip: Once you've imported the JSON into a variable, check the timezone's name to make sure it seems like it got the right part of the world!*

In [ ]:
url = 'https://api.darksky.net/forecast/'
api_key = 'a58aefb6c38668940bd73244663ac26f'
place = {'lat': '52.366667', 'long' : '9.716667'} #Hanover, Germany
options = ['?units=si']

In [ ]:
place_url = url+api_key+"/"+place['lat']+","+place['long']+options[0] #Could've done this with less steps but easier to change now

In [ ]:
place_data = requests.get(place_url).json()

In [ ]:
print(place_data['timezone'])

## 2) What's the current wind speed? How much warmer does it feel than it actually is?

- *Tip: You can do this by browsing through the dictionaries, but it might be easier to read the documentation*
- *Tip: For the second half: it **is** one temperature, and it **feels** a different temperature. Calculate the difference.*

In [ ]:
print(place_data['currently']['windSpeed'])

In [ ]:
print(abs(place_data['currently']['apparentTemperature'] - place_data['currently']['temperature'])) # Delta was zero for me though, weather apparently IS as FEELS

## 3) Today's forecast is the FIRST daily forecast. For the place you decided on above, how much of the moon is currently visible?

- *Tip: This does **not** use the `currently` key.*
- *Tip: It might be helpful to save the forecast into its own variable*

In [ ]:
place_daily = place_data['daily']['data'][0]

In [ ]:
print("There's {} per cent of the moon visible tonight.".format(place_daily['moonPhase']*100))

## 4) What's the difference between the high and low temperatures for today?

In [ ]:
print("The difference between the highest and lowest temperature today is {} degree".format(round(abs(place_daily['temperatureHigh'] - place_daily['temperatureLow']),2)))

## 5) Go through the daily forecasts, printing out the next week's worth of predictions.

I'd like to know the **high temperature** for each day, and whether it's **hot, warm, or cold** (based on what temperatures you think are hot, warm or cold).

- *Tip: Is it returning F or C? You can change them by adding `?units=us` or `?units=si` onto the end of the URL.*
- *Tip: You'll need to use an `if` statement to say whether it is hot, warm or cold.*

In [ ]:
week_temp = [day['temperatureHigh'] for day in place_data['daily']['data']]

In [ ]:
print("Next week's weather:")
for day in week_temp:
    if day > 26.5: 
        weather = 'hot'
    elif day > 12: 
        weather = 'warm'
    else:
        weather = 'cold'
    print(weather+":", day, "degrees C.")

# 6) What will be the hottest day in the next week? What is the high temperature on that day?

It's okay if your result looks like `1453363200`. If you want to be extra-fancy, though, format it as a **real date** - the weird number is called a **UNIX timestamp**, which might help in googling.

- *Tip: It's okay to skip this if it seems too difficult!*

In [ ]:
high_day = {'time' : 0, 'hi' : 0.0}

In [ ]:
for day in place_data['daily']['data']:
    if float(day['temperatureHigh']) > high_day['hi']:
        high_day['time'] = int(day['time'])
        high_day['hi'] = day['temperatureHigh']

In [ ]:
high_day['time'] = str(datetime.datetime.fromtimestamp((high_day['time'])).strftime('%m/%d'))#UNIX time conversion

In [ ]:
print("Next week\'s warmest day will be {day} with a maximum of {deg} degree C.".format(day = high_day['time'], deg = high_day['hi']))

## 7) What's the weather looking like for the next 24+ hours in Miami, Florida?

I'd like to know the temperature for every hour, and if it's going to have cloud cover of more than 50% say "{temperature} and cloudy" instead of just the temperature. 


- *Tip: Be sure to remember how latitude and longitude are done with positive and negative numbers*
- *Tip: How do you represent "50%" for cloud cover?*

In [ ]:
url = 'https://api.darksky.net/forecast/'
api_key = 'a58aefb6c38668940bd73244663ac26f'
place = {'lat': '25.775278', 'long' : '-80.208889'} #Miami, Florida
options = ['?units=si']

In [ ]:
place_url = url+api_key+"/"+place['lat']+","+place['long']+options[0]
place_data = requests.get(place_url).json()

In [ ]:
next24temp = [hour['temperature'] for hour in place_data['hourly']['data']]
next24cloud = [hour['cloudCover'] for hour in place_data['hourly']['data']]
hourcounter =list(range(1,48))

In [ ]:
print("The hourly forecast for Miami for the next two days:")
for hour_t, hour_c, count in zip(next24temp, next24cloud, hourcounter):
    if hour_c > .5:
        sky = "cloudy"
    else:
        sky = "not cloudy"
    print("In count h it will be {temp:02} degree C. and the sky will be {sky}.".format(h = count, temp = hour_t, sky = sky))

# 8) For the next 24-ish hours in Miami, what percent of the time is the temperature above 85 degrees?

If they give you 48 hours instead of 24, that's okay.

- *Tip: You might want to read up on [looping patterns](http://jonathansoma.com/lede/foundations-2017/classes/data%20structures/looping-patterns/)*

In [ ]:
h_above_30 = len([hour for hour in next24temp if hour > 30]) #85 F is roughly 30 C

In [ ]:
perc_h_above_30 = round(h_above_30 / len(next24temp)*100, 2)

In [ ]:
print("In the next 2 days, it'll be above 30 C. ", perc_h_above_30, "per cent of the time.")

## 9) What was the temperature in Central Park on Christmas Day, 2012? How about 2007? 2005? How far back does the API allow you to go?

- *Tip: You'll need to use UNIX time, which is the number of seconds since January 1, 1970. Google can help you convert a normal date!*
- *Tip: You'll want to use Forecast.io's "time machine" API, [documentation here](https://darksky.net/dev/docs/time-machine)*

In [ ]:
# Time stamp 25.12.2012, noon
# 1356436800
#
# Time stamp 25.12.2007, noon
# 1198584000
#
# Central Park is at 40.782222, -73.965278
#
# Api scheme: https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]

In [ ]:
url = 'https://api.darksky.net/forecast/'
api_key = 'a58aefb6c38668940bd73244663ac26f'
place = {'lat': '40.782222', 'long' : '-73.965278'} # Central Park NYC
options = ['?units=si', '?exclude=currently', '?exclude=minutely', '?exclude=hourly']
url_pre = url+api_key+"/"+place['lat']+","+place['long']+","

In [ ]:
unix_xmas_2012 = '1356436800'
unix_xmas_2007 = '1198584000'

xmas12place_data = requests.get(url_pre+unix_xmas_2012+options[0]).json()
xmas07place_data = requests.get(url_pre+unix_xmas_2007+options[0]).json()

In [ ]:
print("The temperature in Central Park on Christmas Day in 2012 was", xmas12place_data['currently']['temperature'], "degree C. and in 2007 it was", xmas07place_data['currently']['temperature'], "degree C.")

In [ ]:
# But how far can we go back!?
lo = -2208988800 # Let's set an arbitrary absolute minimum that we believe is earlier then the earliest date available: 1/1/1900 or UNIX time -2208988800
hi = 1198584000 # this is X-Mas 2007, we know that this works from the last Q. The oldest available date has to be somewhere in between the 2.
dif = round((abs(hi-lo)))
r = requests.get(url_pre+str(lo))
status = r.status_code
print(status) # Test run: We get Error 400, now we know that 1/1/1900 is not available


In [ ]:
while abs(hi-lo) > 86400: # We cut the difference between lo/high in half, until we have a diff of 604800: That is one week, we can adjust the precision here but I hit the API max before when I tried to find the first availbale hour.
        print("Dif: {dif}\nStatus:{status}\nHi/Lo: {hi, lo}\n".format(dif = dif, status = status, hi = hi, lo = lo )) # This is just for checking
        r = requests.get(url_pre+str(lo))
        status = r.status_code
        if status == 200: # if we get back 200, we must go lower
            hi = lo
            lo = lo - dif
        else: # if do not get back 200, we must go higher
            lo = lo + dif
        dif = round(dif/2)

In [ ]:
oldest = str(datetime.datetime.fromtimestamp((lo)).strftime('%m/%d/%y'))#UNIX time conversion
print("The oldest avaible weather info is from the week of", oldest)